In [ ]:
#
# ---------------------------------------------------------------------------------
#  FFFFF   A   QQQQ      BBBB   OOO  TTTTT
#  F      A A  Q  Q      B  B  O   O   T
#  FFF   AAAAA Q  Q      BBBB  O   O   T
#  F     A   A Q  Q      B  B  O   O   T
#  F     A   A  QQ Q     BBBB   OOO    T
# 
#  「初めてのFAQ言語モデル構築」ラボへようこそ！
#
#  このノートブックでは、完全な言語モデルをゼロから構築し、
#  簡単なFAQチャットボットとしてトレーニングします。
#
# ---------------------------------------------------------------------------------
#

# =================================================================================
#  ✅ パート1：セットアップとデータ準備
# =================================================================================
#
#  まず、データを準備する必要があります。AIモデルの「燃料」は、質問と回答を含む
#  テキストファイルです。また、必要なライブラリもインポートします。
#
# ---------------------------------------------------------------------------------



In [ ]:
# --- ステップ1.1：ライブラリのインポート ---
# ニューラルネットワークの構築にはPyTorchを、UIにはipywidgetsを使用します。

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# --- ステップ1.2：データセットの作成と読み込み ---
# 実際のワークショップでは、受講者にプロンプトテンプレートを使用してこのテキストファイルを
# 生成させ、「faq.txt」として保存してもらいます。この例では、ここで定義します。
#
# ----------------- あなたのFAQデータをここに記述 -----------------
#
#  手順：
#  1. ワークショップで提供されたプロンプトテンプレートを使用して、FAQコンテンツを生成します。
#  2. 生成されたテキストを以下の三重引用符で囲まれた文字列に貼り付けます。
#  3. ファイルがこのノートブックと同じディレクトリに保存されていることを確認してください。
#

In [ ]:
faq_text = """
Q: What are the store hours?
A: Our store is open from 9 AM to 8 PM, Monday to Saturday.

Q: What is the return policy?
A: You can return any item within 30 days of purchase with a valid receipt.

Q: Do you offer gift wrapping?
A: Yes, we offer complimentary gift wrapping for all in-store purchases.

Q: Where are you located?
A: We are located at 123 Main Street, Anytown, USA.

Q: Can I place an order online?
A: Yes, you can place an order through our website at www.example-store.com.

Q: What payment methods do you accept?
A: We accept all major credit cards, debit cards, and mobile payments.

Q: Is there parking available?
A: Yes, there is a free parking lot available for all our customers behind the store.

Q: Do you have a loyalty program?
A: Yes, you can sign up for our free loyalty program to earn points on every purchase.
"""

In [ ]:
# --- ステップ1.2：ファイルからのデータセットの読み込み ---
# 受講者が作成した`faq.txt`ファイルを読み込みます。
#`faq.txt`ファイルからデータを読み込みます。
# ↓ この部分をコメントアウトまたは削除します ↓
#with open('faq.txt', 'r', encoding='utf-8') as f:
#    faq_text = f.read()

In [ ]:
# ----------------------------------------------------------

# 扱っているデータを見てみましょう

In [ ]:
print("--- Sample of our dataset ---")
print(faq_text[:200])
print("-----------------------------\n")

In [ ]:
# --- ステップ1.3：語彙の作成 ---
# 私たちのモデルは文字を理解できません。数字しか理解できません。そのため、「語彙」を作成し、
# 一意の各文字をそれぞれの一意の整数にマッピングする必要があります。

In [ ]:
chars = sorted(list(set(faq_text)))
vocab_size = len(chars)

print(f"Our vocabulary contains {vocab_size} unique characters.")
print(f"Vocabulary: {''.join(chars)}\n")

In [ ]:
# 文字から整数へ（stoi）、整数から文字へ（itos）のマッピングを作成します

In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [ ]:
# エンコードおよびデコード関数を定義します

In [ ]:
encode = lambda s: [stoi[c] for c in s if c in stoi] # エンコーダー：文字列を受け取り、整数のリストを出力します
decode = lambda l: ''.join([itos[i] for i in l]) # デコーダー：整数のリストを受け取り、文字列を出力します

In [ ]:
# テストしてみましょう

In [ ]:
test_string = "store hours"
encoded_string = encode(test_string)
decoded_string = decode(encoded_string)
print(f"Original: '{test_string}'")
print(f"Encoded: {encoded_string}")
print(f"Decoded: '{decoded_string}'\n")

In [ ]:
# --- ステップ1.4：データセットのトークン化 ---
# これで、テキストデータセット全体を単一の数値シーケンスに変換します。
# PyTorchは、「テンソル」というデータ構造を使用して数値を扱います。

In [ ]:
data = torch.tensor(encode(faq_text), dtype=torch.long)
print(f"Dataset shape: {data.shape}")
print(f"First 100 tokens: {data[:100]}\n")

In [ ]:
# --- ステップ1.5：データをトレーニングセットと検証セットに分割する ---
# データの大部分をモデルのトレーニング（学習）に使用し、一部を検証
# （単に暗記するだけでなく、正しく学習しているかを確認するため）に使用します。

In [ ]:
n = int(0.9 * len(data)) # 最初の90%をトレーニング、残りを検証用とします
train_data = data[:n]
val_data = data[n:]

In [ ]:
# =================================================================================
#  ✅ パート2：コンテキストとバッチの理解
# =================================================================================
#
#  言語モデルは、テキストの塊（コンテキスト）を見て、次の文字を
#  予測しようとすることで学習します。
#
# ---------------------------------------------------------------------------------

In [ ]:
# --- ステップ2.1：コンテキストサイズの定義 ---
# `block_size`は、モデルが見ることができるコンテキストの最大長です。

In [ ]:
block_size = 64
print(f"A single training example (x): {train_data[:block_size].tolist()}")
print(f"The target for each character in x (y): {train_data[1:block_size+1].tolist()}\n")

In [ ]:
# --- ステップ2.2：バッチ作成関数の定義 ---
# モデルを「バッチ」と呼ばれる小さなランダムなデータの塊でトレーニングします。
# これにより、トレーニングプロセスがより効率的で安定します。

In [ ]:
batch_size = 32 # 並行して処理する独立したシーケンスの数は？

def get_batch(split):
    # 入力xとターゲットyの小さなデータバッチを生成します
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
# サンプルバッチを見てみましょう

In [ ]:
xb, yb = get_batch('train')
print("--- Sample Batch ---")
print(f"Inputs (xb) shape: {xb.shape}")
print(f"Targets (yb) shape: {yb.shape}")
print("--------------------\n")

In [ ]:
# =================================================================================
#  ✅ パート3：Transformerモデルの構築（ゼロから！）
# =================================================================================
#
#  ここが最もエキサイティングな部分です！GPTのようなモデルの基盤である
#  Transformerアーキテクチャのコアコンポーネントを構築します。
#
# ---------------------------------------------------------------------------------

In [ ]:
# --- ハイパーパラメータ ---
# これらはモデルの設定です。後でこれらを試すことができます！

In [ ]:
n_embd = 128       # 各文字の埋め込みサイズ
n_head = 4         # アテンションヘッドの数
n_layer = 4        # トランスフォーマーブロックの数
dropout = 0.2      # 過学習を防ぐための正則化手法

In [ ]:
# -----------------------

# --- ステップ3.1：自己アテンションヘッド ---
# これは基本的なコンポーネントです。「アテンションヘッド」により、モデルはコンテキスト内の
# 他の文字を見て、次の文字を予測するために最も重要な文字を決定できます。

In [ ]:
class Head(nn.Module):
    """ 自己アテンションの1つのヘッド """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # アテンションスコア（「親和性」）を計算します
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # 値の加重集計を実行します
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [ ]:
# --- ステップ3.2：マルチヘッドアテンション ---
# モデルをより強力にするために、複数のアテンションヘッドを並行して使用し、
# その結果を組み合わせます。

In [ ]:
class MultiHeadAttention(nn.Module):
    """ 並列のマルチヘッド自己アテンション """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
# --- ステップ3.3：フィードフォワードネットワーク ---
# アテンションメカニズムの後、各文字の表現は、収集された情報を処理するために
# 単純なニューラルネットワークを通過します。

In [ ]:
class FeedForward(nn.Module):
    """ 単純な線形層とそれに続く非線形性 """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# --- ステップ3.4：トランスフォーマーブロック ---
# これで、アテンションとフィードフォワードコンポーネントを単一の「トランスフォーマーブロック」に
# 組み合わせます。実際のLLMは、これらのブロックを多数積み重ねたものです。

In [ ]:
class Block(nn.Module):
    """ トランスフォーマーブロック：通信とその後の計算 """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# --- ステップ3.5：完全な言語モデル ---
# 最後に、すべてを組み立てて完全な言語モデルを作成します！

In [ ]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # 各トークンは、ルックアップテーブルから次のトークンのロジットを直接読み取ります
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # 最終レイヤーノルム
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idxとtargetsは両方とも整数の(B,T)テンソルです
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idxは現在のコンテキストにおけるインデックスの(B, T)配列です
        for _ in range(max_new_tokens):
            # idxを最後のblock_sizeトークンに切り詰めます
            idx_cond = idx[:, -block_size:]
            # 予測を取得します
            logits, loss = self(idx_cond)
            # 最後のタイムステップのみに焦点を合わせます
            logits = logits[:, -1, :] # (B, C)になります
            # ソフトマックスを適用して確率を取得します
            probs = F.softmax(logits, dim=-1) # (B, C)
            # 分布からサンプリングします
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # サンプリングされたインデックスを実行中のシーケンスに追加します
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
# モデルのインスタンスを作成しましょう！

In [ ]:
model = LanguageModel()
print("Language Model created successfully!")


# =================================================================================
#  ✅ パート4：モデルのトレーニング
# =================================================================================
#
#  これで、データをモデルに供給して学習させます。このプロセスでは、モデルにデータの
#  バッチを表示し、その予測がどれだけ「間違っている」か（「損失」）を計算し、
#  内部パラメータをわずかに調整して改善します。
#
# ---------------------------------------------------------------------------------

In [ ]:
# --- ステップ4.1：オプティマイザの作成 ---
# オプティマイザは、モデルのパラメータを調整するアルゴリズムです。
# AdamWは人気があり効果的な選択肢です。

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# --- ステップ4.2：トレーニングループ ---
# このループは、設定されたステップ数だけ実行されます。各ステップで、データのバッチを取得し、
# モデルに予測を求め、モデルを更新します。
#
# 注：これには数分かかります！
#

In [ ]:
max_iters = 5000 # トレーニングステップの数は？（多いほど良いですが、時間がかかります）
eval_interval = 500 # 検証損失をチェックする頻度は？

print("\n--- Starting Training ---")
for iter in range(max_iters):

    # 時々、トレーニングセットと検証セットの損失を評価します
    if iter % eval_interval == 0:
        # コードの繰り返しを避けるために、損失を推定する関数を作成します
        @torch.no_grad()
        def estimate_loss():
            out = {}
            model.eval()
            for split in ['train', 'val']:
                losses = torch.zeros(200)
                for k in range(200):
                    X, Y = get_batch(split)
                    logits, loss = model(X, Y)
                    losses[k] = loss.item()
                out[split] = losses.mean()
            model.train()
            return out
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # データのバッチをサンプリングします
    xb, yb = get_batch('train')

    # 損失を評価します
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("--- Training Complete! ---\n")

In [ ]:
# =================================================================================
#  ✅ パート5：FAQボットで回答を生成！
# =================================================================================
#
#  いよいよ本番です！トレーニング済みのモデルを使って質問に答えさせましょう。
#  「プロンプト」として質問を与え、モデルが何を生成するか見てみましょう。
#
# ---------------------------------------------------------------------------------

# --- ステップ5.1：生成関数 ---
# ボットと対話するための簡単な関数を書きましょう。

In [ ]:
def ask_bot(question):
    """
    質問文字列を受け取り、モデルを使って回答を生成します。
    """
    # モデル用のプロンプトを準備します
    prompt = f"Q: {question}\nA:"
    print(prompt, end='') # プロンプトを改行なしで表示します

    # プロンプトをエンコードしてテンソルを作成します
    context = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0)

    # 回答を生成します
    generated_output = model.generate(context, max_new_tokens=100)[0].tolist()

    # 結果をデコードして表示します
    answer = decode(generated_output)
    # 生成された部分だけが必要なので、回答がどこから始まるかを見つけます
    answer_part = answer[len(prompt):]
    print(answer_part.split('Q:')[0].split('\n\n')[0]) # 新しい質問が始まったら表示を停止します

In [ ]:
# --- ステップ5.2：テストしてみましょう！ ---
# これがデータベースではないことを証明するために、トレーニングデータとは
# 少し異なる質問をしてみましょう。これにより、モデルが学習したパターンから
# 一般化する能力をテストします。

In [ ]:
print("--- Ask the FAQ Bot ---")
ask_bot("what are the store hours?") # 小文字を使用
ask_bot("do you offer gift wrapping") # 小文字で、疑問符なし
ask_bot("return policy?") # より曖昧で部分的な質問
print("-----------------------")

In [ ]:
# =================================================================================
#  ✅ パート6：インタラクティブなFAQボット！
# =================================================================================
#
#  それでは、このノートブック上で簡単なユーザーインターフェースを作成し、
#  ボットと対話できるようにしましょう。
#
# ---------------------------------------------------------------------------------

In [ ]:
# --- ステップ6.1：ユーザーインターフェースの構築 ---
# `ipywidgets`ライブラリを使用して、テキストボックスとボタンを作成します。

In [ ]:
# ウィジェットの作成

In [ ]:
question_input = widgets.Text(
    value='What are the store hours?',
    placeholder='Type your question here',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='Ask Bot',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to get an answer',
    icon='question-circle'
)

output_area = widgets.Output()

In [ ]:
# --- ステップ6.2：ボタンクリックアクションの定義 ---
# この関数は、「ボットに質問」ボタンをクリックするたびに実行されます。

In [ ]:
def on_button_clicked(b):
    with output_area:
        output_area.clear_output() # 前の回答をクリア
        question = question_input.value

        # モデル用のプロンプトを準備
        prompt = f"Q: {question}\nA:"
        print(prompt, end='')

        # プロンプトをエンコードしてテンソルを作成
        context = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0)

        # 回答を生成
        generated_output = model.generate(context, max_new_tokens=100)[0].tolist()

        # 結果をデコードして表示
        answer = decode(generated_output)
        answer_part = answer[len(prompt):]
        # 新しい質問を開始したり、ループに陥ったりした場合は、印刷を停止します
        final_answer = answer_part.split('Q:')[0].split('\n\n')[0]
        print(final_answer)

In [ ]:
# ボタンを関数にリンクします

In [ ]:
submit_button.on_click(on_button_clicked)

In [ ]:
# --- ステップ6.3：ボットを表示！ ---
# UI要素を表示します。質問を入力してボタンをクリックしてください！

In [ ]:
print("--- Interactive FAQ Bot ---")
display(question_input, submit_button, output_area)

In [ ]:
# --- 最終的な考察と次のステップ ---
#
# おめでとうございます！インタラクティブなUIを備えた言語モデルを構築し、デプロイしました。
#
# これは「シード」プロジェクトです。完璧ではありませんが、コアコンセプトを実証しています。
# 改善するためには、次のことができます：
#   1. faq.txtファイルにさらにデータを追加する。
#   2. トレーニングの反復回数を増やす（`max_iters`を増やす）。
#   3. ハイパーパラメータ（例：`n_embd`、`n_head`、`n_layer`）を試す。
#   4. UIとエラーハンドリングを改善する。
#